In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)

In [3]:
dir(iris)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [4]:
df['flower']=iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
df['flower']=df['flower'].apply(lambda x:iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
#train test split
inputs=df.drop('flower',axis='columns')
target=df.flower
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inputs,target,test_size=.2)

In [7]:
#calling svm and manually tuning
from sklearn.svm import SVC
sv=SVC(kernel='rbf',C=30,gamma='auto')

sv.fit(x_train,y_train)
sv.score(x_test, y_test)
#from sklearn.model_selection import GridSearchCV


0.9

In [17]:
#tunning with the help of kfold
from sklearn.model_selection import cross_val_score
SVC(kernel='rbf',C=10,gamma='auto')

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [9]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),x_train,y_train,cv=5)

array([0.96      , 0.95833333, 0.95833333, 0.95833333, 1.        ])

In [10]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),x_train,y_train,cv=5)

array([0.96      , 0.95833333, 1.        , 0.95833333, 1.        ])

In [11]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),x_train,y_train,cv=5)

array([0.96      , 0.95833333, 1.        , 0.95833333, 1.        ])

In [12]:
cross_val_score(SVC(kernel='linear',C=20,gamma='auto'),inputs,target,cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

In [13]:
#above method is lenghty so using loop for the same
import numpy as np
score={}
kernels = ['rbf', 'linear']
C = [1,10,20]
for kval in kernels:
    for cval in C:
        cv_scores=cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),inputs,target,cv=5)
        score[kval + '_' + str(cval)] = np.average(cv_scores)
score

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [14]:
#approach 3 is using gridsearch cv
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_
        

{'mean_fit_time': array([0.0010047 , 0.00020051, 0.00142112, 0.00189495, 0.00128808,
        0.0021811 ]),
 'std_fit_time': array([2.50464363e-05, 4.01020050e-04, 1.85926156e-03, 2.02279039e-03,
        2.09108634e-03, 2.31301493e-03]),
 'mean_score_time': array([0.00049901, 0.00157609, 0.00021214, 0.00042815, 0.00019946,
        0.0004416 ]),
 'std_score_time': array([0.00041222, 0.00202704, 0.00042429, 0.00052628, 0.00039892,
        0.00054525]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [15]:
#for visulizing it in a better way using pandas
df1=pd.DataFrame(clf.cv_results_)
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001005,0.000025,0.000499,0.000412,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000201,0.000401,0.001576,0.002027,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001421,0.001859,0.000212,0.000424,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001895,0.002023,0.000428,0.000526,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001288,0.002091,0.000199,0.000399,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.002181,0.002313,0.000442,0.000545,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,5


In [16]:
df1=df1[['params','mean_test_score']]
df1

,params,mean_test_score
0,"{'C': 1, 'kernel': 'rbf'}",0.980000
1,"{'C': 1, 'kernel': 'linear'}",0.980000
2,"{'C': 10, 'kernel': 'rbf'}",0.980000
3,"{'C': 10, 'kernel': 'linear'}",0.973333
4,"{'C': 20, 'kernel': 'rbf'}",0.966667
5,"{'C': 20, 'kernel': 'linear'}",0.966667


In [31]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [52]:
#here we will use multiple model and will be doing their parameter tuning
#for this we will be creating dictionaries
model_params={
    'svm':{
        'model':svm.SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'randomforest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
    'Logisticregression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,10,20]
        }
    }
}


In [53]:
#now play the for loop for finding the best suitable algo for iris

scores=[]
for model_names,mp in model_params.items():
    clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(inputs,target)
    scores.append({
        'model':model_names,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })

In [59]:
df1=pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [60]:
df1

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,randomforest,0.966667,{'n_estimators': 5}
2,Logisticregression,0.966667,{'C': 10}


In [66]:
for model_names,m in model_params.items():
    print(m)

{'model': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear']}}
{'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'params': {'n_estimators': [1, 5, 10]}}
{'model': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100